# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis!

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# create a spark session
spark = SparkSession.builder \
    .appName("Linear Regression with PySpark MLlib") \
    .getOrCreate()
spark

In [ ]:
data = spark.read.csv("cruise_ship_info.csv", header=True, inferSchema=True)
data.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [ ]:
# assemble the input features into a single feature vector using the VectorAssembler class
'''
features=feature vector, which is the assembled vector of multiple columns,
crew=target variable
'''
assembler = VectorAssembler(
    inputCols=["Age", "Tonnage", "passengers", "length", "cabins", "passenger_density"],
    outputCol="features")

data = assembler.transform(data)


In [ ]:
data.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [ ]:
final_data = data.select("features", "crew")
final_data.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [ ]:
# train test split
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
train_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[4.0,220.0,54.0,1...| 21.0|
|[5.0,86.0,21.04,9...|  8.0|
|[5.0,122.0,28.5,1...|  6.7|
|[5.0,133.5,39.59,...|13.13|
|[5.0,160.0,36.34,...| 13.6|
|[6.0,30.276999999...| 3.55|
|[6.0,93.0,23.94,9...|11.09|
|[6.0,110.23899999...| 11.5|
|[6.0,112.0,38.0,9...| 10.9|
|[6.0,113.0,37.82,...| 12.0|
|[7.0,89.6,25.5,9....| 9.87|
|[7.0,116.0,31.0,9...| 12.0|
|[7.0,158.0,43.7,1...| 13.6|
|[8.0,77.499,19.5,...|  9.0|
|[8.0,91.0,22.44,9...| 11.0|
|[9.0,59.058,17.0,...|  7.4|
|[9.0,81.0,21.44,9...| 10.0|
|[9.0,85.0,19.68,9...| 8.69|
|[9.0,90.09,25.01,...| 8.69|
|[9.0,105.0,27.2,8...|10.68|
+--------------------+-----+
only showing top 20 rows



In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                25|
|   mean|            8.6732|
| stddev|2.7322452915749227|
|    min|              3.55|
|    max|              13.6|
+-------+------------------+



In [ ]:
test_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[5.0,115.0,35.74,...| 12.2|
|[6.0,30.276999999...| 3.55|
|[6.0,90.0,20.0,9....|  9.0|
|[6.0,158.0,43.7,1...| 13.6|
|[8.0,110.0,29.74,...| 11.6|
|[9.0,88.5,21.24,9...| 10.3|
|[9.0,116.0,26.0,9...| 11.0|
|[10.0,86.0,21.14,...|  9.2|
|[11.0,90.0,22.4,9...| 11.0|
|[11.0,90.09,25.01...| 8.48|
|[11.0,91.0,20.32,...| 9.99|
|[11.0,108.977,26....| 12.0|
|[11.0,138.0,31.14...|11.85|
|[12.0,50.0,7.0,7....| 4.45|
|[12.0,91.0,20.32,...| 9.99|
|[13.0,76.0,18.74,...|  8.5|
|[14.0,63.0,14.4,7...| 5.61|
|[17.0,74.137,19.5...|  7.6|
|[19.0,55.451,12.6...| 5.88|
|[20.0,55.451,12.6...| 5.57|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# building a linear regression model
lr = LinearRegression(featuresCol="features", labelCol="crew", predictionCol="predicted_value")
lr_model = lr.fit(train_data)

In [ ]:
model_results=lr_model.evaluate(train_data)
print(model_results.r2)
print(f'rootMeanSquaredError: {model_results.rootMeanSquaredError}')
print(f'meanAbsoluteError: {model_results.meanAbsoluteError}')
print(f'explainedVariance: {model_results.explainedVariance}')
print("Coefficients: ", lr_model.coefficients)
print("Intercept: ", lr_model.intercept)
print('residuals')
model_results.residuals.show()


0.922275480836511
rootMeanSquaredError: 1.0040242046607488
meanAbsoluteError: 0.6243091780871937
explainedVariance: 11.961647070376811
Coefficients:  [-0.010903003418631511,0.013324492286938255,-0.14854321913524848,0.3980699041625792,0.803022083753731,0.0011692417052758689]
Intercept:  -0.5882123307479032
residuals
+--------------------+
|           residuals|
+--------------------+
| 0.28735243897406804|
|  -1.358436565649498|
| 0.28137652305021454|
|  0.3389312792518222|
|    -1.5600611430706|
|-0.43402545527774183|
|   0.561603276721268|
|  0.4194358696398268|
| -0.1595040556845202|
|  0.4422316957454058|
| -0.9765595666683584|
| -0.6087196146569553|
|-0.27261237236537816|
|  0.2559849418906559|
|  0.8975222913458616|
|-0.07892431617566498|
|  0.4730275977306455|
| -0.5071023876001686|
| -0.7656083985759796|
| -0.4692911007053695|
+--------------------+
only showing top 20 rows



In [ ]:
predictions = lr_model.transform(test_data)

In [ ]:
predictions.show()

+--------------------+-----+------------------+
|            features| crew|   predicted_value|
+--------------------+-----+------------------+
|[5.0,115.0,35.74,...| 12.2|11.503208271909205|
|[6.0,30.276999999...| 3.55|3.9840254552777417|
|[6.0,90.0,20.0,9....|  9.0| 9.727816566550349|
|[6.0,158.0,43.7,1...| 13.6|13.935264463325144|
|[8.0,110.0,29.74,...| 11.6|12.142415881066224|
|[9.0,88.5,21.24,9...| 10.3| 9.748050239854877|
|[9.0,116.0,26.0,9...| 11.0|11.274281488526832|
|[10.0,86.0,21.14,...|  9.2| 9.657409356194098|
|[11.0,90.0,22.4,9...| 11.0|10.045892873770686|
|[11.0,90.09,25.01...| 8.48| 9.080472674887076|
|[11.0,91.0,20.32,...| 9.99| 9.209183752260302|
|[11.0,108.977,26....| 12.0|11.160753206540775|
|[11.0,138.0,31.14...|11.85|13.120186382254719|
|[12.0,50.0,7.0,7....| 4.45| 4.655906440637441|
|[12.0,91.0,20.32,...| 9.99| 9.198280748841672|
|[13.0,76.0,18.74,...|  8.5|  8.61369958049956|
|[14.0,63.0,14.4,7...| 5.61| 6.885482762896711|
|[17.0,74.137,19.5...|  7.6|  8.83792193

In [ ]:
evaluator = RegressionEvaluator(labelCol="crew", predictionCol="predicted_value", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))

Root Mean Squared Error (RMSE) on test data: 0.683


In [ ]:
evaluator_r2 = RegressionEvaluator(labelCol="crew", predictionCol="predicted_value", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R-squared (R2) on test data: {:.3f}".format(r2))

R-squared (R2) on test data: 0.935


In [ ]:
lr_model.save('linear_regression_model')